# Introduction to Pandas 3

Advanced Pandas syntax and joining.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('fivethirtyeight')

## 1. Name Whose Popularity has Changed the Most

Let's plot the change in popularity of a name to see if we find any trends.

Run the cell below.

In [ ]:
babynames_nc = pd.read_csv("babynames_nc.csv", index_col = 0)

**Example 1.1.** First we need to define change in popularity. For the purposes of this demonstration, let’s stay simple and use the absolute maximum and minimum difference (ammd).

To make sure we understand this quantity, let's consider the name Jordan.

Write a function named `ammd`that will compute the difference between the times the name Jordan was given the most and the least.

In [ ]:
def ammd(series):
    return max(series) - min(series)

name = "Jordan"
counts = babynames_nc[babynames_nc.Name == name]["Count"]

ammd(counts)

### 1.1. For Loop

**Example 1.1.1.** Ideallly we would like to do this for all the names in the table. In Foundations fo Data Science we learned to use the `for` loop. We could use a dictionary to store the name (i.e. key) and count (i.e. value) as pairs.

In [ ]:
# Build dictionary where entry i is the ammd function for the given name
# For example, ammd["Jordan"] should be 489

ammd_of_babyname_counts = {}

name = "Jordan"

# Select the first 10 unique names
for name in babynames_nc["Name"].unique()[0:10]:
    counts_of_current_name = babynames_nc[babynames_nc["Name"] == name]["Count"]
    ammd_of_babyname_counts[name] = ammd(counts_of_current_name)

# Convert to a series
ammd_of_babyname_counts = pd.Series(ammd_of_babyname_counts) 
ammd_of_babyname_counts

### 1.2. Use `groupby` and `agg`

**Example 1.2.1.** We can use the `groupby.agg` operation, which allows us to simply and efficiently compute what we want.

In [ ]:
babynames_nc.groupby("Name").agg(max).head()

**Example 1.2.2.** Using the `ammd` functions for `agg()`.

In [ ]:
babynames_nc.groupby("Name").agg(ammd).head()

**Note:** The result includes both a Year and Count column. The Count column is what we want, namely the ammd for the name in that row. To check your understanding, try to figure out what the Year column represents.

To understand how `groupby` works, consider the visual diagram below. The `groupby` function clusters rows from the original dataframe into groups (which I call subframes). The `agg` function then condenses each subframe into a single representative row using the provided function $f$.

<center><img src="groupbyagg.png"></center>

**Example 1.2.3.** Let's actually build this DataFrame and play with it interactively to understand `groupby` better.

In [ ]:
df = pd.DataFrame(dict(x=[3,1,4,1,5,9,2,5,6], y=[12,7,3,2,7,3,8,6,7]), index=list('ABCABCACB') )
df

In [ ]:
df_gb = df.groupby(df.index)
df_gb

In [ ]:
df_gb.groups

In [ ]:
df_gb.groups['A']

In [ ]:
df_gb.get_group('A')

In [ ]:
df_gb.agg(ammd)

## 2. Some Additional Groupby Examples

**Example 2.1.** Test your understanding, by trying to interpret the result of the code below.

In [ ]:
babynames_nc.groupby("Year").agg(ammd).plot();

**Example 2.2.** Let's look at the intermediate dataframes for each year.

In [ ]:
for i, (k, v) in enumerate(babynames_nc.groupby("Year")):
    if i >= 3: break
    print(f"Year: {k}")
    display(v.head())

**Example 2.3.** For reference, the first 5 values from the plot above are:

In [ ]:
babynames_nc.groupby("Year").agg(ammd).head()

In [ ]:
elections = pd.read_csv("elections.csv")
elections.head()

**Example 2.2.** We have to be careful when using aggregation functions. For example, the code below might be misinterpreted to say that Woodrow Wilson ran for election in 2016. Why is this happening?

In [ ]:
elections.groupby("Party").agg(max).head(10)

**Example 3.1.** Inspired by above, try to predict the results of the groupby operation shown.

<center><img src="groupbyagg1.png"></center>

**Example 3.2.** What goes in the first box with **??**, the second box with **??**, and the third box with **??**?

In [ ]:
df = pd.DataFrame(dict(x=[3,1,4,1,5,9,2,5,6], 
                      y=['ak', 'tx', 'fl', 'hi', 'mi', 'ak', 'ca', 'sd', 'nc']), 
                      index=list('ABCABCACB') )
df

**Example 3.3.** Check your solutions by running the cell below.

In [ ]:
df.groupby(ds.index).agg(max)

**Example 4.1.** Next we'll write code that properly returns the best result by each party. That is, each row should show the Year, Candidate, Popular Vote, Result, and % for the election in which that party saw its best results (rather than mixing them as in the example above), here's what the first rows of the correct output should look like:

<center><img src="groupbyagg2.png"></center>

**Example 4.2.** You'll soon discover that with Pandas' rich tool set, there's typically more than one way to get to the same answer. Each approach has different tradeoffs in terms of readability, performance, memory consumption, complexity and more. It will take some experience for you to develop a sense of which approach is better for each problem, but you should in general try to think if you can at least envision a different solution to a given problem, especially if you find your current solution to be particularly convoluted or hard to read.

Here's a couple ways of obtaining the same result (in each case we only show the top part with head()). The first approach uses groupby but finds the location of the maximum value via the `idxmax()` method (look up its documentation!). We then index and sort by party to match the requested formatting:

In [ ]:
elections_sorted_by_percent = elections.sort_values('%', ascending = False)
elections_sorted_by_percent.groupby("Party").agg(lambda x : x.iloc[0]).head()

In [ ]:
elections_sorted_by_percent = elections.sort_values("%", ascending = False)
elections_sorted_by_percent.groupby("Party").first().head()

In [ ]:
best_per_party = elections.loc[elections.groupby('Party')['%'].idxmax()]  
best_per_party.set_index('Party').sort_index().head()                  

In [ ]:
best_per_party2 = elections.sort_values('%').drop_duplicates(['Party'], keep = 'last')
best_per_party2.set_index('Party').sort_index().head()                            

## 3. Join Review

In this section we'll briefly review joining tables as discussed in Data 8 (click [here](https://www.inferentialthinking.com/chapters/08/4/Joining_Tables_by_Columns.html) to review joining).

Often data is spread across two tables. Joining provides us with a way to naturally combine related tables.

Let's start by reading data from the given zip file. To showcase how to do read data that is inside zip files, we're going to use the `zipfile` module. Doing so will allow us to avoid needing to unzip the data. 


**Example 3.1.** Running the cell below, we see that the zip file contains `elections.csv` and `presidents.csv`.

In [ ]:
import zipfile

join_demo_filename = "join_demo_data.zip"
my_zip = zipfile.ZipFile(join_demo_filename, 'r')
list_names = [f.filename for f in my_zip.filelist]
list_names

**Example 3.2.** We could call `my_zip.extractall()` to unzip the files, but we won't. Instead, we'll read directly from the zip file itself.

In [ ]:
with my_zip.open("elections.csv") as f:
    elections = pd.read_csv(f)
    
elections.head()

In [ ]:
with my_zip.open("presidents.csv") as f:
    presidents = pd.read_csv(f)
    
presidents.head()

**Example 3.3.** To join tables `df` and `df2`, we call the function `df.merge(df2)`. Merge is just the word that the authors of pandas picked for joining tables. I don't know why.

**Note:** Unfortunately, Pandas also has a function called `df.join`. This is a limited version of merge. For the sake of generality, we will only use merge in this class.

We can use the merge function to combine these two tables: 

In [ ]:
elections.merge(presidents, 
            how = "inner",
            left_on = "Candidate", right_on = "President")

Notice that:

1. The output dataframe only contains rows that have names in both tables. For example, presidents before 1824 do not appear in the joined table because there was no popular vote before 1824.

2. The name Andrew Jackson occurred three times in the election table and shows up three times in the output.

3. Grover Cleveland occurs six times! Twice for every election he was in. This is because he appears three times in the elections table and twice in the presidents table. This results in 3 x 2 = 6 combinations.

4. Several presidents are missing because their names are not an exact match. For example, John F. Kennedy is "John Kennedy" in the elections table and "John F. Kennedy" in the presidents table.

**Example 3.4.** With the caveats above in mind, this merged DataFrame is handy because we can use it to plot, e.g. the age of each president when they were elected.

In [ ]:
joined = elections.merge(presidents, 
            how = "inner",
            left_on = "Candidate", right_on = "President")

winners = joined.query("Result == 'win'").copy()
winners["Birthyear"] = winners["Date of birth"].str.split(',').str[1].map(int)
winners["Age"] = winners["Year"] - winners["Birthyear"]
sns.lmplot(data=winners, x="Year", y="Age");

#### 3.1. How could we fix the duplicate Grover Cleveland?

**Example 3.1.1.** We could group by name/candidate and take only the first:

In [ ]:
(
    elections.merge(presidents, 
            how = "inner",
            left_on = "Candidate", 
            right_on = "President").
    groupby(['Candidate', 'Year']).first().reset_index()
)

#### 3.2. Right Join

The above join was an inner join. 


**Example 3.2.1.** What if we wanted to keep all of the presidents and leave missing data for years when there was no popular vote? In this case we'd do a "right" join, where we make sure to include **every** row from our right dataframe, in this case presidents.

In [ ]:
elections.merge(presidents, 
            how = "right",
            left_on = "Candidate", right_on = "President")

#### 3.3. Left Join

**Example 3.3.1.** Similarly, if we also want to include candidates not in the presidents table (e.g. because they had never won), we can use a "left" join.

In [ ]:
elections.merge(presidents, 
            how = "left",
            left_on = "Candidate", right_on = "President")

#### 3.4. Outer Join

**Example 3.4.1.** If we wanted to keep both, we can instead do an "outer join".

In [ ]:
elections.merge(presidents, 
            how = "outer",
            left_on = "Candidate", right_on = "President")